# $\textit{Predicción lineal}$
$\textit{By Agustín Luís Gullino - Gonzalo Ezequiel Linares}$

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as sp
import helper as hp
import importlib
from numpy.fft import fft, rfft
from numpy.fft import fftshift, fftfreq, rfftfreq
importlib.reload(hp)

fs = 16000  # Frecuencia de muestreo

In [5]:
fs, signal, path = hp.getRecordedSignals()

/home/gullino18/Documents/Github/PASA_TP3/helper.py:18: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs1, aSignal = io.wavfile.read(path[0])
/home/gullino18/Documents/Github/PASA_TP3/helper.py:19: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs2, shSignal = io.wavfile.read(path[1])


In [7]:
from arch.unitroot import ADF
from arch.unitroot import PhillipsPerron

''' 
Para el test, se tiene en cuenta que las hipotesis son:

H0: La serie contiene raiz unitaria.
H1: La serie es estacionaria.

Si el p-valor esta por encima del critico, la hipotesis nula no puede ser rechazada, indicando que 
la serie no es estacionaria.
'''

norm1 = signal["a"]
norm2 = signal["sh"]
max_lags_test=len(signal["a"])//10

adf = ADF(norm1, max_lags=max_lags_test)
print(adf.summary().as_text())
print("Dado que P-value = 0, podemos concluir que la señal de aaa es estacionaria.")
print('\n//////////////////////////////////////\n')

adf = ADF(norm2, max_lags=max_lags_test)
print(adf.summary().as_text())
print("Dado que P-value = 0, podemos concluir que la señal de shh estacionaria.")

pp = PhillipsPerron(norm1)
print(pp.summary().as_text())
print("Dado que P-value = 0, podemos concluir que la señal de aaa estacionaria.")
print('\n//////////////////////////////////////\n')

pp = PhillipsPerron(norm2)
print(pp.summary().as_text())
print("Dado que P-value = 0, podemos concluir que la señal de shh estacionaria.")

   Augmented Dickey-Fuller Results   
Test Statistic                 -4.030
P-value                         0.001
Lags                              538
-------------------------------------

Trend: Constant
Critical Values: -3.43 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.
Dado que P-value = 0, podemos concluir que la señal de aaa es estacionaria.

//////////////////////////////////////

   Augmented Dickey-Fuller Results   
Test Statistic                -13.106
P-value                         0.000
Lags                              164
-------------------------------------

Trend: Constant
Critical Values: -3.43 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.
Dado que P-value = 0, podemos concluir que la señal de shh estacionaria.
     Phillips-Perron Test (Z-tau)    
Test Statistic                -12.

In [ ]:
Ms = np.linspace(1, 30, 30, dtype=int)
jos_a = np.zeros(len(Ms))
jos_sh = np.zeros(len(Ms))
a_a = np.zeros((len(Ms), len(Ms)+1))
a_sh = np.zeros((len(Ms), len(Ms)+1))

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
for i, M in enumerate(Ms):
    wo, jo, NMSE = hp.get_optimal_params(signal["a"][:-1], signal["a"], M)
    jos_a[i] = jo
    a_a[i][:i+2] = np.insert(-wo, 0, 1) 

    wo, jo, NMSE = hp.get_optimal_params(signal["sh"][:-1], signal["sh"], M)
    jos_sh[i] = jo
    a_sh[i][:i+2] = np.insert(-wo, 0, 1) 

axs[0].set_xlabel('M')
axs[0].set_ylabel('MSE mínimo')
axs[0].plot(Ms, jos_a)

axs[1].set_xlabel('M')
axs[1].set_ylabel('MSE mínimo')
axs[1].plot(Ms, jos_sh)

In [16]:
e_foa = np.convolve(a_a[28], signal["a"])

NameError: name 'a_a' is not defined

In [ ]:
plt.figure()
plt.plot(e_foa)

In [ ]:
box = rfft(SquareW, N*10)
freq = rfftfreq(N*10, 1/fs)
response = 10*np.log10((np.abs(A)+1e-1) / abs(A).max())
responsebox = 10*np.log10((np.abs(box)+1e-18) / abs(box).max())

axs[1].set_title("Frec. response")
axs[1].plot(freq, responsebox, label='Box')
axs[1].plot(freq, response, label='blackman')
axs[1].set_xlabel("frecuency")
axs[1].set_ylabel("dB")
axs[1].set_xscale('log')
axs[1].set_ylim(-80, 1)
plt.legend()